# `ProteinWorkshop` Tutorial, Part 3 - Adding a New Dataset
![Datasets](../docs/source/_static/box_datasets.png)

In [ ]:
%load_ext autoreload
%autoreload 2
#%load_ext blackcellmagic

The `ProteinWorkshop` encomposses several datasets for pre-training and finetuning. However, you may want to add your own dataset to the `ProteinWorkshop` to fulfill a specific use case. This tutorial will show you how to do that.

To add your custom dataset to the `ProteinWorkshop`, you just have to follow the following 4-step procedure (created files in brackets):

1. Create a new subclass of the `ProteinDataModule` class (`my_new_dataset.py`)
2. Create a new data config file to accompany the custom `MyNewDataModule` (`my_new_dataset.yaml`)
3. Compose and instantiate your config for pre-training or finetuning using your dataset
4. Use your custom dataset for a pre-training or finetuning task

### 1. Create a new subclass of the `ProteinDataModule` class

Reference the `CATHDataModule` below (i.e., `proteinworkshop/datasets/cath.py`) to fill out a custom `proteinworkshop/datasets/my_new_dataset.py` in a similar style. 

In [ ]:
"""
class CATHDataModule(ProteinDataModule):
    def __init__(
        self,
        path: str,
        batch_size: int,
        format: str = "mmtf",
        pdb_dir: Optional[str] = None,
        pin_memory: bool = True,
        in_memory: bool = False,
        num_workers: int = 16,
        dataset_fraction: float = 1.0,
        transforms: Optional[Iterable[Callable]] = None,
        ) -> None:
        super().__init__()

        self.data_dir = Path(path)
        self.raw_dir = self.data_dir / "raw"
        self.processed_dir = self.data_dir / "processed"
        if not os.path.exists(self.data_dir):
            os.makedirs(self.data_dir)

        if transforms is not None:
            self.transform = self.compose_transforms(
                omegaconf.OmegaConf.to_container(
                    transforms,
                    resolve=True
                    )
                )
        else:
            self.transform = None

        self.in_memory = in_memory

        self.batch_size = batch_size
        self.pin_memory = pin_memory
        self.num_workers = num_workers
        self.format = format
        self.pdb_dir = pdb_dir

        self.dataset_fraction = dataset_fraction
        self.excluded_chains: List[str] = self.exclude_pdbs()

    def download(self):
        self.download_chain_list()

    def parse_labels(self):
        pass

    def exclude_pdbs(self):
        return []

    def download_chain_list(self):  # sourcery skip: move-assign
        URL = "http://people.csail.mit.edu/ingraham/graph-protein-design/data/cath/chain_set_splits.json"
        if not os.path.exists(self.data_dir / "chain_set_splits.json"):
            logger.info("Downloading dataset index file...")
            wget.download(URL, str(self.data_dir / "chain_set_splits.json"))
        else:
            logger.info("Found existing dataset index")

    @functools.lru_cache
    def parse_dataset(self) -> Dict[str, List[str]]:
        fpath = self.data_dir / "chain_set_splits.json"

        with open(fpath, "r") as file:
            data = json.load(file)

        self.train_pdbs = data["train"]
        logger.info(f"Found {len(self.train_pdbs)} chains in training set")
        logger.info("Removing obsolete PDBs from training set")
        self.train_pdbs = [pdb for pdb in self.train_pdbs if pdb[:4] not in self.obsolete_pdbs.keys()]
        logger.info(f"{len(self.train_pdbs)} remaining training chains")

        logger.info(f"Sampling fraction {self.dataset_fraction} of training set")
        fraction = int(self.dataset_fraction * len(self.train_pdbs))
        self.train_pdbs = random.sample(self.train_pdbs, fraction)

        self.val_pdbs = data["validation"]
        logger.info(f"Found {len(self.val_pdbs)} chains in validation set")
        logger.info("Removing obsolete PDBs from validation set")
        self.val_pdbs = [pdb for pdb in self.val_pdbs if pdb[:4] not in self.obsolete_pdbs.keys()]
        logger.info(f"{len(self.val_pdbs)} remaining validation chains")

        self.test_pdbs = data["test"]
        logger.info(f"Found {len(self.test_pdbs)} chains in test set")
        logger.info("Removing obsolete PDBs from test set")
        self.test_pdbs = [pdb for pdb in self.test_pdbs if pdb[:4] not in self.obsolete_pdbs.keys()]
        logger.info(f"{len(self.test_pdbs)} remaining test chains")
        return data

    def train_dataset(self):
        if not hasattr(self, "train_pdbs"):
            self.parse_dataset()
        pdb_codes = [pdb.split(".")[0] for pdb in self.train_pdbs]
        chains = [pdb.split(".")[1] for pdb in self.train_pdbs]

        return ProteinDataset(
            root=str(self.data_dir),
            pdb_dir=self.pdb_dir,
            pdb_codes=pdb_codes,
            chains=chains,
            transform=self.transform,
            format=self.format,
            in_memory=self.in_memory
        )

    def val_dataset(self) -> ProteinDataset:
        if not hasattr(self, "val_pdbs"):
            self.parse_dataset()

        pdb_codes = [pdb.split(".")[0] for pdb in self.val_pdbs]
        chains = [pdb.split(".")[1] for pdb in self.val_pdbs]

        return ProteinDataset(
            root=str(self.data_dir),
            pdb_dir=self.pdb_dir,
            pdb_codes=pdb_codes,
            chains=chains,
            transform=self.transform,
            format=self.format,
            in_memory=self.in_memory
        )

    def test_dataset(self) -> ProteinDataset:
        if not hasattr(self, "test_pdbs"):
            self.parse_dataset()
        pdb_codes = [pdb.split(".")[0] for pdb in self.test_pdbs]
        chains = [pdb.split(".")[1] for pdb in self.test_pdbs]

        return ProteinDataset(
            root=str(self.data_dir),
            pdb_dir=self.pdb_dir,
            pdb_codes=pdb_codes,
            chains=chains,
            transform=self.transform,
            format=self.format,
            in_memory=self.in_memory
        )

    def train_dataloader(self) -> ProteinDataLoader:
        if not hasattr(self, "train_ds"):
            self.train_ds = self.train_dataset()
        return ProteinDataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            shuffle=True,
            drop_last=True,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
        )

    def val_dataloader(self) -> ProteinDataLoader:
        if not hasattr(self, "val_ds"):
            self.val_ds = self.val_dataset()
        return ProteinDataLoader(
            self.val_ds,
            batch_size=self.batch_size,
            shuffle=False,
            drop_last=True,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
        )

    def test_dataloader(self) -> ProteinDataLoader:
        if not hasattr(self, "test_ds"):
            self.test_ds = self.test_dataset()
        return ProteinDataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            shuffle=False,
            drop_last=True,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
        )
"""

### 2. Create a new data config file to accompany the custom `MyNewDataModule`

Reference the `CATH` config below (i.e., `configs/dataset/cath.yaml`) to fill out a custom `configs/dataset/my_new_dataset.yaml`. This config file sets the actual values of the parameters of your datamodule. This includes dataset options like the path where the dataset is stored, but also datamodule options for creating dataloaders like the number of workers or the batch size.

In [ ]:
"""
datamodule:
  _target_: "proteinworkshop.datasets.cath.CATHDataModule"
  path: ${env.paths.data}/cath/ # Directory where the dataset is stored
  pdb_dir: ${env.paths.data}/pdb/ # Directory where raw PDB/mmtf files are stored
  format: "mmtf" # Format of the raw PDB/MMTF files
  num_workers: 4 # Number of workers for dataloader
  pin_memory: True # Pin memory for dataloader
  batch_size: 32 # Batch size for dataloader
  dataset_fraction: 1.0 # Fraction of the dataset to use
  transforms: ${transforms} # Transforms to apply to dataset examples
  num_classes: 23 # Number of classes
"""

### 3. Compose and instantiate your config for pre-training or finetuning using your dataset

Now we need to use the created config file in our code. To do this, we use `Hydra`, a library that helps with managing configuration options via `.yaml` files.

In the following code block, we initialize Hydra and then compose the `cfg` object which we will later use to perform downstream or pre-training tasks. We can pass `hydra.compose` various overrides in order to customize our setup. We can specify for example:
- the encoder to use
- the task to perform later on
- the dataset to use (here our new custom dataset)
- the features that are used
- which auxiliary test should be performed (if any)

In [ ]:
# Misc. tools
import os

# Hydra tools
import hydra

from hydra.compose import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from proteinworkshop.constants import HYDRA_CONFIG_PATH
from proteinworkshop.utils.notebook import init_hydra_singleton

version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")

GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(config_name="train", overrides=["encoder=schnet", "task=inverse_folding", "dataset=my_new_dataset", "features=ca_angles", "+aux_task=none"], return_hydra_config=True)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

After configuring your config object via `hydra.compose`, you can instantiate your datamodule via `hydra.utils.instantiate`.

In [ ]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

datamodule = hydra.utils.instantiate(cfg.dataset.datamodule)
datamodule.setup("train")
dl = datamodule.train_dataloader()

for i in dl:
    print(i)
    break

### 4. Use your custom dataset for a pre-training or finetuning task

Instead of instantiating your datamodule to use it directly, you can make use of the infrastructure that `ProteinWorkshop` provides in order to directly use the config object for training or finetuning a model, depending on what your goal is.

In [ ]:
from proteinworkshop.finetune import finetune
from proteinworkshop.train import train_model

# train_model(cfg)  # Pre-train a model using the selected data
# finetune(cfg)  # Fine-tune a model using the selected data

When we instantiated the config, we specified `ca_angles` as feature context. However, we can easily reconfigure the custom dataset to use side-chain atom context as you can see in the following code block.

In [ ]:
version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")

GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(config_name="train", overrides=["encoder=schnet", "task=inverse_folding", "dataset=my_new_dataset", "features=ca_sc", "+aux_task=none"], return_hydra_config=True)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

Now we can verify that side-chain torsions are available as feature inputs. This procedure is not only valid for the features, but can be used to reconfigure the encoder, the task and all the other options specified in the config.

In [ ]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

datamodule = hydra.utils.instantiate(cfg)
datamodule.setup("train")
dl = datamodule.train_dataloader()

for i in dl:
    print(i)
    break

### 5. Wrapping up

Have any additional questions about adding your custom dataset to the `ProteinWorkshop`? [Create a new issue](https://github.com/a-r-j/ProteinWorkshop/issues/new/choose) on our [GitHub repository](https://github.com/a-r-j/ProteinWorkshop). We would be happy to work with you to add your new dataset to the repository!